In [2]:
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import cv2 as cv
from sudoku import Sudoku

In [3]:
img = cv.imread("2.jpg", cv.IMREAD_COLOR)
img = imutils.resize(img, width = 600)

In [4]:
cv.imshow("", img)
cv.waitKey(0)

-1

In [7]:
def boundaries(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (7,7), 3)
    threshold = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11,2)
    threshold = cv.bitwise_not(threshold)
    cv.imshow("", threshold)
    cv.waitKey(0)
    contour = cv.findContours(threshold.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contour = imutils.grab_contours(contour)
    contour = sorted(contour, key = cv.contourArea, reverse=True)
    count = None
    
    for c in contour:
        length = cv.arcLength(c, True)
        
        approx = cv.approxPolyDP(c, 0.02 * length, True)
        
        if len(approx) == 4:
            count = approx
            print(length)
            print(approx)
            break
        
    if count is None:
        print("No puzzle found")
        
    img = image.copy()
    cv.drawContours(img, [count], -1, (0,255,0),2)
    cv.imshow("",img)
    cv.waitKey(0)
    print(count)
    puzzle = four_point_transform(image, count.reshape(4,2))
    warped = four_point_transform(gray, count.reshape(4,2)) 
    cv.imshow("",puzzle)
    cv.waitKey(0)
    return (puzzle, warped)

In [8]:
color, grayscale = boundaries(img)

2293.213203072548
[[[ 12  16]]

 [[ 14 588]]

 [[585 587]]

 [[584  15]]]
[[[ 12  16]]

 [[ 14 588]]

 [[585 587]]

 [[584  15]]]


In [9]:
def extract(cell):
    threshold = cv.threshold(cell, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
    print(threshold.shape)
    threshold = clear_border(threshold)
    #cv.imshow("",threshold)
    #cv.waitKey(0)
    contour = cv.findContours(threshold.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contour = imutils.grab_contours(contour)
    img = cell.copy()
    cv.drawContours(img, contour, -1, (0,255,0),2)
    #cv.imshow("",img)
    #cv.waitKey(0)
    if len(contour) == 0:
        return None
    
    c = max(contour, key = cv.contourArea)
    mask = np.zeros(threshold.shape).astype("int8")
    cv.drawContours(mask, [c], -1, 255, -1)
    #cv.imshow("",mask)
    #cv.waitKey(0)
    (height, width) = threshold.shape
    filled = cv.countNonZero(mask)/float(height*width)
    
    if filled < 0.03:
        return None
    
    digit = cv.bitwise_and(threshold, threshold, mask=mask)
    #cv.imshow("", digit)
    #cv.waitKey(0)
    return digit

In [10]:
extract(grayscale)

(572, 572)


In [11]:
grey.shape[0]

NameError: name 'grey' is not defined

In [12]:

digit_classifier = load_model("digit_classifier")
(colored, grey) = boundaries(img)
board = np.zeros((9,9), dtype = "int")
X_size = grey.shape[1] // 9
y_size = grey.shape[0] // 9

cells = []

for y in range(0,9):
    row = []
    for x in range(0,9):
        startX = x * X_size
        startY = y * y_size
        endX = (x+1) * X_size
        endY = (y+1) * y_size
        row.append((startX, startY, endX, endY))
        print((startX, startY, endX, endY))
        cell = grey[startY:endY, startX:endX]
        digit = extract(cell)
        
        if digit is not None:
            area = cv.resize(digit, (28,28))
            area = area.astype("float") / 255.0
            area = img_to_array(area)
            area = np.expand_dims(area, axis = 0)
            prediction = digit_classifier.predict(area).argmax(axis=1)[0]
            print(prediction)
            board[y,x] = prediction
    cells.append(row)
        

2293.213203072548
[[[ 12  16]]

 [[ 14 588]]

 [[585 587]]

 [[584  15]]]
[[[ 12  16]]

 [[ 14 588]]

 [[585 587]]

 [[584  15]]]
(0, 0, 63, 63)
(63, 63)
(63, 0, 126, 63)
(63, 63)
(126, 0, 189, 63)
(63, 63)
4
(189, 0, 252, 63)
(63, 63)
6
(252, 0, 315, 63)
(63, 63)
7
(315, 0, 378, 63)
(63, 63)
8
(378, 0, 441, 63)
(63, 63)
9
(441, 0, 504, 63)
(63, 63)
(504, 0, 567, 63)
(63, 63)
(0, 63, 63, 126)
(63, 63)
(63, 63, 126, 126)
(63, 63)
3
(126, 63, 189, 126)
(63, 63)
(189, 63, 252, 126)
(63, 63)
(252, 63, 315, 126)
(63, 63)
(315, 63, 378, 126)
(63, 63)
(378, 63, 441, 126)
(63, 63)
(441, 63, 504, 126)
(63, 63)
5
(504, 63, 567, 126)
(63, 63)
(0, 126, 63, 189)
(63, 63)
2
(63, 126, 126, 189)
(63, 63)
(126, 126, 189, 189)
(63, 63)
(189, 126, 252, 189)
(63, 63)
(252, 126, 315, 189)
(63, 63)
5
(315, 126, 378, 189)
(63, 63)
(378, 126, 441, 189)
(63, 63)
(441, 126, 504, 189)
(63, 63)
(504, 126, 567, 189)
(63, 63)
1
(0, 189, 63, 252)
(63, 63)
5
(63, 189, 126, 252)
(63, 63)
(126, 189, 189, 252)
(63, 63)


In [13]:
print(board)

[[0 0 4 6 7 8 9 0 0]
 [0 3 0 0 0 0 0 5 0]
 [2 0 0 0 5 0 0 0 1]
 [5 0 0 4 0 6 0 0 9]
 [9 0 0 3 0 7 0 0 4]
 [3 0 2 0 0 0 8 0 6]
 [4 1 0 0 0 0 0 9 2]
 [0 9 0 0 0 0 0 6 0]
 [0 0 5 7 1 9 3 0 0]]


In [18]:
solution = Sudoku(3,3, board = board.tolist())
solution = solution.solve()
solution.board[0][0]

1

In [139]:
test = color
color = test

In [140]:

for (cell, sol) in zip(cells, solution.board):
    for(box, digit) in zip(cell, sol):
        X1, Y1, X2, Y2 = box
        X = int((X2-X1) * 0.33)
        Y = int((Y2-Y1) * -0.2)
        X += X1
        Y += Y2
        print(box)
        cv.putText(color, str(digit), (X,Y), cv.FONT_HERSHEY_SIMPLEX,0.9, (0,255,0),2)
cv.imshow("", color)
cv.waitKey(0)

(0, 0, 63, 63)
(63, 0, 126, 63)
(126, 0, 189, 63)
(189, 0, 252, 63)
(252, 0, 315, 63)
(315, 0, 378, 63)
(378, 0, 441, 63)
(441, 0, 504, 63)
(504, 0, 567, 63)
(0, 63, 63, 126)
(63, 63, 126, 126)
(126, 63, 189, 126)
(189, 63, 252, 126)
(252, 63, 315, 126)
(315, 63, 378, 126)
(378, 63, 441, 126)
(441, 63, 504, 126)
(504, 63, 567, 126)
(0, 126, 63, 189)
(63, 126, 126, 189)
(126, 126, 189, 189)
(189, 126, 252, 189)
(252, 126, 315, 189)
(315, 126, 378, 189)
(378, 126, 441, 189)
(441, 126, 504, 189)
(504, 126, 567, 189)
(0, 189, 63, 252)
(63, 189, 126, 252)
(126, 189, 189, 252)
(189, 189, 252, 252)
(252, 189, 315, 252)
(315, 189, 378, 252)
(378, 189, 441, 252)
(441, 189, 504, 252)
(504, 189, 567, 252)
(0, 252, 63, 315)
(63, 252, 126, 315)
(126, 252, 189, 315)
(189, 252, 252, 315)
(252, 252, 315, 315)
(315, 252, 378, 315)
(378, 252, 441, 315)
(441, 252, 504, 315)
(504, 252, 567, 315)
(0, 315, 63, 378)
(63, 315, 126, 378)
(126, 315, 189, 378)
(189, 315, 252, 378)
(252, 315, 315, 378)
(315, 315,

-1

In [ ]:
len(cells[0])

In [ ]:
np.asarray(cells).shape

In [ ]:
for (cell, sol) in zip(cells, solution.board):
    for(box, digit) in zip(cell, sol):
        print(box, digit)